# `megamerge`

The aim is to create a merge index / overlap list as fast as possible.

This may involve `numpy`/`scipy`, or we may delve into Rust.

The aim is to complete the part of the merge task that takes exponential time, such that python can perform the aggregations which take linear time.

It looks like the longest road segment is on H006 its 1050km long (or 3200km if you ignore POEs). Any road segment longer than this is probably not practically useful.

In [2]:
import pandas as pd
import numpy as np

In [28]:
segments = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to"],
    data=[
        ["H001", "L",   0, 100],
        ["H001", "L", 100, 200],
        ["H001", "L", 200, 300],
        ["H001", "L", 300, 400],

        ["H001", "R",   0, 100],
    ]
)

data = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to", "measure", "category"],
    data=[                             # overlaps lengths
        ["H001", "L", 50,  140,  1.0, "A"],  # 50  40   0  0    [50, 40,  0,  0]
        ["H001", "L", 140, 160,  2.0, "B"],  #  0  20   0  0    [ 0, 20,  0,  0]
        ["H001", "L", 160, 180,  3.0, "B"],  #  0  20   0  0    [ 0, 20,  0,  0]
        ["H001", "L", 180, 220,  4.0, "B"],  #  0  20  20  0    [ 0, 20, 20,  0]
        ["H001", "L", 220, 240,  5.0, "C"],  #  0   0  20  0    [ 0,  0, 20,  0]
        ["H001", "L", 240, 260,  5.0, "C"],  #  0   0  20  0    [ 0,  0, 20,  0]
        ["H001", "L", 260, 280,  6.0, "D"],  #  0   0  20  0    [ 0,  0, 20,  0]
        ["H001", "L", 280, 300,  7.0, "E"],  #  0   0  20  0    [ 0,  0, 20,  0]
        ["H001", "L", 300, 320,  8.0, "F"],  #  0   0     20    [ 0,  0,  0, 20]

        ["H001", "R",  10,  80,  9.0, "G"],  #  0   0     20
        ["H001", "R",  80, 120, 10.0, "H"],  #  0   0     20
    ]
)

expected_result = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to",  "measure longest value",  "category longest value"],
    data=[
        ["H001", "L",   0, 100,  1.0,  "A"],
        ["H001", "L", 100, 200,  1.0,  "B"],
        ["H001", "L", 200, 300,  5.0,  "C"],
        ["H001", "L", 300, 400,  8.0,  "F"],

        ["H001", "R",   0, 100,  9.0,  "G"],
    ]
)


## Old Merge Method

Using `dtimsprep` library which accuratly reproduces the results from the old merge macro.

In [29]:
import dtimsprep.merge as merge
result = merge.on_slk_intervals(
    segments,
    data,
    ["road", "cwy"],
    [
        merge.Action('measure',  rename="measure longest value",  aggregation=merge.Aggregation.KeepLongest()),
        merge.Action('category', rename="category longest value", aggregation=merge.Aggregation.KeepLongest()),
    ]
)
pd.testing.assert_frame_equal(result, expected_result)

ModuleNotFoundError: No module named 'dtimsprep'

## MegaMerge

Now lets try do this FASTER

In [30]:
from typing import Union

# preprocess the index on target segmentation to reduce the number of columns we need to work with
def get_categorical_index(segmentation:pd.DataFrame, categories:list[str]) -> Union[pd.Index, pd.MultiIndex]:
    """
    categories should not be in the index
    """
    return (
        segmentation
        .loc[:, list(categories)]
        .drop_duplicates()
        .set_index(categories)
        .index
    )

In [31]:
# Squish all the categorical indecies
seg_idx = get_categorical_index(segments, ["road", "cwy"])
seg_idx

MultiIndex([('H001', 'L'),
            ('H001', 'R')],
           names=['road', 'cwy'])

In [32]:
# modify the segmentation to use the squished index
segments_with_compacted_index = segments.copy()
segments_with_compacted_index = segments_with_compacted_index.drop(columns=["road","cwy"])
segments_with_compacted_index["compacted_category_index"] = seg_idx.get_indexer(segments.loc[:, ["road", "cwy"]])
segments_with_compacted_index=segments_with_compacted_index.set_index("compacted_category_index")
segments_with_compacted_index

,slk_from,slk_to
compacted_category_index,,
0,0,100
0,100,200
0,200,300
0,300,400
1,0,100


In [33]:
# modify the data to use the squished index
data_with_compacted_index = data.copy()
data_with_compacted_index = data_with_compacted_index.drop(columns=["road","cwy"])
data_with_compacted_index["compacted_category_index"] = seg_idx.get_indexer(data.loc[:, ["road", "cwy"]])
data_with_compacted_index=data_with_compacted_index.set_index("compacted_category_index")
data_with_compacted_index

,slk_from,slk_to,measure,category
compacted_category_index,,,,
0,50,140,1.0,A
0,140,160,2.0,B
0,160,180,3.0,B
0,180,220,4.0,B
0,220,240,5.0,C
0,240,260,5.0,C
0,260,280,6.0,D
0,280,300,7.0,E
0,300,320,8.0,F


In [34]:
# now lets look at the matching chunk from both dataframes
D = data_with_compacted_index.loc[0,:]
S = segments_with_compacted_index.loc[0,:]
display(D)
display(S)


,slk_from,slk_to,measure,category
compacted_category_index,,,,
0,50,140,1.0,A
0,140,160,2.0,B
0,160,180,3.0,B
0,180,220,4.0,B
0,220,240,5.0,C
0,240,260,5.0,C
0,260,280,6.0,D
0,280,300,7.0,E
0,300,320,8.0,F


,slk_from,slk_to
compacted_category_index,,
0,0,100
0,100,200
0,200,300
0,300,400


In [10]:
overlap_minimum = np.maximum(
    D["slk_from"].values,
    np.expand_dims(S["slk_from"].values,1),
)
overlap_minimum

array([[ 50, 140, 160, 180, 220, 240, 260, 280, 300],
       [100, 140, 160, 180, 220, 240, 260, 280, 300],
       [200, 200, 200, 200, 220, 240, 260, 280, 300],
       [300, 300, 300, 300, 300, 300, 300, 300, 300]], dtype=int64)

In [11]:
overlap_maximum = np.minimum(
    D["slk_to"].values,
    np.expand_dims(S["slk_to"].values,1),
)
np.transpose(overlap_maximum)

array([[100, 140, 140, 140],
       [100, 160, 160, 160],
       [100, 180, 180, 180],
       [100, 200, 220, 220],
       [100, 200, 240, 240],
       [100, 200, 260, 260],
       [100, 200, 280, 280],
       [100, 200, 300, 300],
       [100, 200, 300, 320]], dtype=int64)

In [12]:
overlap = np.maximum(0, overlap_maximum - overlap_minimum)
np.transpose(overlap)
#overlap = overlap.astype("f8")
#overlap[overlap==-100]=np.nan
#overlap

array([[50, 40,  0,  0],
       [ 0, 20,  0,  0],
       [ 0, 20,  0,  0],
       [ 0, 20, 20,  0],
       [ 0,  0, 20,  0],
       [ 0,  0, 20,  0],
       [ 0,  0, 20,  0],
       [ 0,  0, 20,  0],
       [ 0,  0,  0, 20]], dtype=int64)

In [13]:

D.reset_index().join(pd.DataFrame(np.transpose(overlap), columns=pd.MultiIndex.from_arrays(S.loc[:,["slk_from",	"slk_to"]].values.transpose())))

C:\Users\guilt\AppData\Local\Temp\ipykernel_9432\3003654395.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  D.reset_index().join(pd.DataFrame(np.transpose(overlap), columns=pd.MultiIndex.from_arrays(S.loc[:,["slk_from",	"slk_to"]].values.transpose())))


,compacted_category_index,slk_from,slk_to,measure,category,"(0, 100)","(100, 200)","(200, 300)","(300, 400)"
0,0,50,140,1.0,A,50,40,0,0
1,0,140,160,2.0,B,0,20,0,0
2,0,160,180,3.0,B,0,20,0,0
3,0,180,220,4.0,B,0,20,20,0
4,0,220,240,5.0,C,0,0,20,0
5,0,240,260,5.0,C,0,0,20,0
6,0,260,280,6.0,D,0,0,20,0
7,0,280,300,7.0,E,0,0,20,0
8,0,300,320,8.0,F,0,0,0,20


## Lets Go Big!

In [67]:
# generate segmentation
big_segments = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to"],
    data   =[
        ["H001", "L",   index*500, (index+1)*500] for index in range(int(3000 / 0.500))
    ]
)
big_segments

,road,cwy,slk_from,slk_to
0,H001,L,0,500
1,H001,L,500,1000
2,H001,L,1000,1500
3,H001,L,1500,2000
4,H001,L,2000,2500
...,...,...,...,...
5995,H001,L,2997500,2998000
5996,H001,L,2998000,2998500
5997,H001,L,2998500,2999000
5998,H001,L,2999000,2999500


In [68]:
big_data = pd.DataFrame(
    columns= ["road", "cwy", "slk_from", "slk_to", "measure"],
    data   = [
        ["H001", "L",   (index)*10, (index+1)*10, index] for index in range(int(3000 / 0.010))
    ]
)
big_data

,road,cwy,slk_from,slk_to,measure
0,H001,L,0,10,0
1,H001,L,10,20,1
2,H001,L,20,30,2
3,H001,L,30,40,3
4,H001,L,40,50,4
...,...,...,...,...,...
299995,H001,L,2999950,2999960,299995
299996,H001,L,2999960,2999970,299996
299997,H001,L,2999970,2999980,299997
299998,H001,L,2999980,2999990,299998


In [69]:
# Squish all the categorical indecies
seg_idx = get_categorical_index(big_segments, ["road", "cwy"])
print(seg_idx)

# modify the segmentation to use the squished index
segments_with_compacted_index = big_segments.copy()
segments_with_compacted_index = segments_with_compacted_index.drop(columns=["road","cwy"])
segments_with_compacted_index["compacted_category_index"] = seg_idx.get_indexer(big_segments.loc[:, ["road", "cwy"]])
segments_with_compacted_index=segments_with_compacted_index.set_index("compacted_category_index")
display(segments_with_compacted_index)

# modify the data to use the squished index
data_with_compacted_index = big_data.copy()
data_with_compacted_index = data_with_compacted_index.drop(columns=["road","cwy"])
data_with_compacted_index["compacted_category_index"] = seg_idx.get_indexer(big_data.loc[:, ["road", "cwy"]])
data_with_compacted_index=data_with_compacted_index.set_index("compacted_category_index")
display(data_with_compacted_index)

D = data_with_compacted_index.loc[0,:]
S = segments_with_compacted_index.loc[0,:]

MultiIndex([('H001', 'L')],
           names=['road', 'cwy'])


,slk_from,slk_to
compacted_category_index,,
0,0,500
0,500,1000
0,1000,1500
0,1500,2000
0,2000,2500
...,...,...
0,2997500,2998000
0,2998000,2998500
0,2998500,2999000


,slk_from,slk_to,measure
compacted_category_index,,,
0,0,10,0
0,10,20,1
0,20,30,2
0,30,40,3
0,40,50,4
...,...,...,...
0,2999950,2999960,299995
0,2999960,2999970,299996
0,2999970,2999980,299997


In [70]:
def overlap(segmentation,data,offset,chunk_size=1000, proximity_threshold=0):
    overlap_minimum = np.maximum(
        segmentation["slk_from"].values[offset:offset+chunk_size],
        np.expand_dims(data["slk_from"].values,1),
    )
    overlap_maximum = np.minimum(
        segmentation["slk_to"].values[offset:offset+chunk_size],
        np.expand_dims(data["slk_to"].values,1),
    )
    return np.maximum(overlap_maximum - overlap_minimum - proximity_threshold, 0)

In [71]:
print(f"processing {len(D):,.0f} datapoints onto {len(S):,.0f} segments is")
print(f"equivalent to checking every pixle in 4k video at 24fps for {len(D)*len(S)*8 / ((1920*4)*(1080*4)*3)/24/60:.1f} minutes")
print(f"We expect it to take at about that long.")

processing 300,000 datapoints onto 6,000 segments is
equivalent to checking every pixle in 4k video at 24fps for 0.1 minutes
We expect it to take at about that long.


In [72]:
chunk_size = 200
num_chunks = int(len(S) / chunk_size)
print("."*num_chunks)
print("")
import time
start_time = time.perf_counter_ns()
last_backspace=0
for i in range(num_chunks):
    overlap(
        segmentation        = S,
        data                = D,
        offset              = i*chunk_size,
        chunk_size          = chunk_size,
        proximity_threshold = 0
    )
    cur_time = time.perf_counter_ns();
    elapsed = (cur_time-start_time)/1e9
    #print("\b" * last_backspace,end="\r")
    #to_print = f"remaining: {(num_chunks-i)*(elapsed/(i+1)):.0f} seconds      "
    #last_backspace = len(to_print)
    print(f"elapsed: {elapsed:.0f} remaining: {(num_chunks-i)*(elapsed/(i+1)):.0f} seconds     ",end="\r")
print(f" " * last_backspace, end="\r")
end_time = time.perf_counter_ns();
print(f"elapsed {(end_time-start_time)/1e9:.0f} seconds                          ")

..............................

elapsed 24 secondsing: 1 seconds      


In [73]:
def overlap(segmentation,data,offset,chunk_size=1000, proximity_threshold=0):
    overlap_minimum = np.maximum(
        segmentation["slk_from"].values[offset:offset+chunk_size],
        np.expand_dims(data["slk_from"].values,1),
    )
    overlap_maximum = np.minimum(
        segmentation["slk_to"].values[offset:offset+chunk_size],
        np.expand_dims(data["slk_to"].values,1),
    )
    return np.maximum(overlap_maximum - overlap_minimum - proximity_threshold, 0)

def merg (segmentation, data, chunk_size, proximity_threshold):
    num_chunks = int(len(segmentation) / chunk_size);
    for offset in range(0, len(segmentation), chunk_size):
        ovlp = overlap(
            segmentation        = segmentation,
            data                = data,
            offset              = offset,
            chunk_size          = chunk_size,
            proximity_threshold = proximity_threshold
        )
        yield (
            offset,
            [
                [
                    list((ind := np.nonzero(item)[0])),
                    list(np.take(item, ind) + proximity_threshold),
                ]   
                for item in np.transpose(ovlp)
            ]
        )

In [75]:
for offset, chunk in merg(S, D, chunk_size=200, proximity_threshold=0):
    print(f"=== chunk {offset}")
    for index, item in enumerate(chunk):
        pass
        # print(f"segmentation index: {index+offset}")
        # print(f"data index:         {','.join(map(str, item[0]))}")
        # print(f"overlap:            {','.join(map(str, item[1]))}")
        # print("")

=== chunk 0
=== chunk 200
=== chunk 400
=== chunk 600
=== chunk 800
=== chunk 1000
=== chunk 1200
=== chunk 1400
=== chunk 1600
=== chunk 1800
=== chunk 2000
=== chunk 2200
=== chunk 2400
=== chunk 2600
=== chunk 2800
=== chunk 3000
=== chunk 3200
=== chunk 3400
=== chunk 3600
=== chunk 3800
=== chunk 4000
=== chunk 4200
=== chunk 4400
=== chunk 4600
=== chunk 4800
=== chunk 5000
=== chunk 5200
=== chunk 5400
=== chunk 5600
=== chunk 5800


In [58]:
for offset, chunk in merg(S, D, chunk_size=2, proximity_threshold=0):
    print(f"=== chunk {offset}")
    print(chunk)
    for index, item in enumerate(chunk):
        print(f"segmentation index: {index+offset}")
        print(f"data index:         {','.join(map(str, item[0]))}")
        print(f"overlap:            {','.join(map(str, item[1]))}")
        print("")

=== chunk 0
[[[0], [50]], [[0, 1, 2, 3], [40, 20, 20, 20]]]
segmentation index: 0
data index:         0
overlap:            50

segmentation index: 1
data index:         0,1,2,3
overlap:            40,20,20,20

=== chunk 2
[[[3, 4, 5, 6, 7], [20, 20, 20, 20, 20]], [[8], [20]]]
segmentation index: 2
data index:         3,4,5,6,7
overlap:            20,20,20,20,20

segmentation index: 3
data index:         8
overlap:            20

